In [ ]:
from __future__ import annotations

import json
import os
import urllib.parse
import urllib.request
import ipykernel
from pathlib import Path


def _as_path(p):
    if not p:
        return None
    try:
        return Path(p).expanduser().resolve()
    except Exception:
        return None


def _papermill_path():
    # Available when papermill is run with --inject-input-path / --inject-paths
    return _as_path(globals().get("PAPERMILL_INPUT_PATH"))


def _vscode_path():
    # Jupyter in VS Code sets this global
    return _as_path(globals().get("__vsc_ipynb_file__"))


def _jpy_session_name_path():
    # Some Jupyter frontends/servers expose the session path via the environment
    return _as_path(os.environ.get("JPY_SESSION_NAME"))


def _ipynbname_path():
    # Optional dependency: pip install ipynbname
    try:
        import ipynbname
        return _as_path(ipynbname.path())
    except Exception:
        return None


def _server_sessions_path():
    """
    Query the running Jupyter server's /api/sessions to map kernel id -> notebook path.
    Works only when attached to a Jupyter server (classic/Lab/etc.).
    """
    # Jupyter 6.x uses notebook.notebookapp; Jupyter 7 uses jupyter_server.serverapp
    try:
        from notebook import notebookapp as serverapp  # type: ignore
    except Exception:
        try:
            from jupyter_server import serverapp  # type: ignore
        except Exception:
            return None

    try:
        cf = Path(ipykernel.get_connection_file()).name  # e.g. kernel-<id>.json
        kid = cf.split("-", 1)[1].split(".")[0]
    except Exception:
        return None

    for s in serverapp.list_running_servers():
        base_url = s.get("url")
        if not base_url:
            continue

        sessions_url = urllib.parse.urljoin(base_url, "api/sessions")

        # Auth: token may be required depending on how the server is started
        token = s.get("token") or ""
        if token:
            parsed = list(urllib.parse.urlparse(sessions_url))
            q = dict(urllib.parse.parse_qsl(parsed[4]))
            q["token"] = token
            parsed[4] = urllib.parse.urlencode(q)
            sessions_url = urllib.parse.urlunparse(parsed)

        try:
            with urllib.request.urlopen(sessions_url, timeout=2) as resp:
                sessions = json.load(resp)

            for sess in sessions:
                if sess.get("kernel", {}).get("id") == kid:
                    nb = sess.get("notebook", {})
                    nb_path = nb.get("path")  # path relative to server root
                    if not nb_path:
                        continue

                    # Jupyter can report different keys depending on version/config
                    root = s.get("root_dir") or s.get("notebook_dir") or ""
                    return _as_path(Path(root) / nb_path)
        except Exception:
            continue

    return None


def get_notebook_path():
    """
    Best-effort notebook file path. Returns None if not discoverable.
    """
    for f in (
        _papermill_path,         # When running headless via papermill
        _vscode_path,            # When in VS Code
        _jpy_session_name_path,  # Opportunistic
        _ipynbname_path,         # Optional dependency
        _server_sessions_path,   # Last resort for interactive server sessions
    ):
        p = f()
        if p and p.suffix.lower() == ".ipynb":
            return p
    return None


def get_notebook_dir() -> Path:
    """
    Best-effort containing directory for the current notebook, else cwd.
    """
    p = get_notebook_path()
    return (p.parent if p else Path.cwd().resolve())

In [ ]:
from pathlib import Path
import os

TRUNCATE_AFTER = "reports"

def get_project_root_folder():
    # Get the notebook folder and create a Path object from it
    notebook_folder = get_notebook_dir()
    path = Path(notebook_folder)

    # Find the part up to and including the final element after which we truncate
    for parent in path.parents:
        if parent.name == TRUNCATE_AFTER:
            return parent

    # Fallback - just return CWD
    return os.getcwd()